In [6]:
import os
import json
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from agent1_logic import build_sbvr_local_search_engine

# 定義 Agent 狀態，嚴格對應你原始的設計
class Agent1State(TypedDict, total=False):
    question: str
    rules: List[Dict]
    raw_response: str
    error: str

In [7]:
search_engine = build_sbvr_local_search_engine(
    index_root=os.path.join("..", "kyc_workflow", "output"),
    sbvr_system_prompt_path="sbvr_local_search_system_prompt.txt",
)

In [8]:
async def agent1_node(state: Agent1State) -> Agent1State:
    """
    Agent 的核心節點，呼叫 search_engine 並處理結果。
    """
    question = state["question"]
    try:
        # 呼叫 search_engine
        result = await search_engine.search(question)
        raw = result.response

        # 解析 JSON
        try:
            data = json.loads(raw)
            rules = data.get("rules", [])
        except json.JSONDecodeError:
            rules = []
        
        # 回傳結果到狀態中
        return {
            "question": question,
            "rules": rules,
            "raw_response": raw,
        }
    except Exception as e:
        # 處理執行期間的錯誤
        return {
            "question": question,
            "rules": [],
            "raw_response": "",
            "error": str(e),
        }

In [9]:
def build_agent1_graph():
    graph = StateGraph(Agent1State)
    graph.add_node("agent1", agent1_node)
    graph.set_entry_point("agent1")
    graph.add_edge("agent1", END)
    return graph.compile()

In [10]:
graph = build_agent1_graph()

# 定義問題
question = (
    "From HKMA SPM g33A, extract all CDD rules that apply to non-account holders"
)

# 執行 Agent
result = await graph.ainvoke({"question": question})

# 顯示結果
print("--- Agent Result ---")
print("Error:", result.get("error"))
print("Rules count:", len(result.get("rules", [])))
print("\nFirst rule sample:")
if result.get("rules"):
    print(json.dumps(result.get("rules")[:1], indent=2, ensure_ascii=False))
else:
    print("No rules found or an error occurred.")

Reached token limit - reverting to previous context state
Reached token limit - reverting to previous context state


--- Agent Result ---
Error: None
Rules count: 3

First rule sample:
[
  {
    "id": "CDD-R-NAH-001",
    "source_reference": "g33a: SUPPLEMENT TO THE GUIDELINE ON PREVENTION OF MONEY LAUNDERING, Section 3. Customer due diligence",
    "clause_reference": "3.10",
    "data_support": "[Data: TextUnits (26)]",
    "subject": "authorized institution",
    "modality": "obligation",
    "condition": "when carrying out transactions exceeding HK$120,000 on behalf of a non-account holder",
    "action": "conduct customer due diligence measures on the non-account holder",
    "action_steps": [
      "identify and verify the direct customer",
      "identify and verify any natural persons representing the customer, including their authority to act",
      "enquire if any beneficial owner exists and take reasonable measures to verify the identity of any such beneficial owner",
      "take reasonable measures to understand the ownership structure if the customer is a corporate",
      "ascertain th